In [1]:
# TFG: Diagnosi del COVID amb Intel·ligència Artificial
# Tutors: Xavier Lladó i Robert Martí
# Autor: Marc Padrós Jiménez
# GEINF, UdG, Curs 21/22

# Primera versió de l'algoritme que dibuixa quadrícules en diferents imatges 
# de radiografies de tòrax 
# Les quadrícules estan dibuixades a l'esquerra i a la dreta de la columna vertebral
# Tant a la part esquerra com a la part dreta, hi ha el mateix nombre de regions 

# He seleccionat 6 imatges del dataset de la carpeta COVID de la web: https://data.mendeley.com/datasets/9xkhgts2s6/3
# He seleccionat les imatges que he cregut que eren més diferents si per tal de comprovar les afectacions que produeix, per exemple, una imatge molt cremada, molt blanca
# respecte una imatge nítida 

In [2]:
# Llegim les imatges que seran processades 

import glob
import cv2 as cv
import numpy as np

path = glob.glob("/Test_images/*")

# Creem una llista buida, que contindrà les matrius d'intensitats associades a les imatges d'entrada
images_ori = [] # BW
images_res = [] # color 


for img in path:
    image_BW = cv.imread(img, cv.IMREAD_GRAYSCALE) # Llegim cada imatge en nivell de grisos 
    images_ori.append(image_BW)
    image_color = cv.imread(img) # Llegim cada imatge en color
    images_res.append(image_color)

In [3]:
def getColumn(image):
    width = image.shape[1] # shape retorna una tupla (height, width, number_of_channels)
    # Obtenim la columna central de la imatge
    center_column = (width // 2) # l'operador // serveix per fer una divisió entera
    # Detectem la regió de la columna vertebral
    start_column = center_column - 50
    end_column = center_column + 50
    vertebral_column = image[:, start_column:end_column]
    return vertebral_column, start_column, end_column

In [4]:
def gridColumn(column, start_column, end_column, nGrids, image_res, ROI):
    # Create n horizontally grids along the vertebral column obtained from rx_image
    # All the grids except the last will have the same size and the last grid will be bigger 
    # and will start from half the column 
    # Paint each grid in RGB rx_image 

    # Paint vertical lines
    image_res[:, start_column+1] = (0, 0, 255) # OpenCV by default reads images as BGR 
    image_res[:, end_column+1] = (0, 0, 255) 

    # Obtenim la fila central 
    center_row = column.shape[0] // 2
    
    # Compute increment from one grid to another
    inc = center_row // nGrids
    # Current row
    j = 0
    # Compute n-1 grids
    for i in range(nGrids-1):
        ROI[i] = column[j : j+inc+1, :]        
        j = j + inc
        # Paint horizontal line
        image_res[j, start_column: end_column+1] = (0, 0, 255)

    # Compute last grid
    ROI[i+1] = column[j:, :]




In [5]:
def gridAroundColumn(start_column, end_column, nGridsHor, nGridsVer, image_ori, image_res, ROI):
    
    idx_row = 0
    nMovements = (nGridsVer - 1) // 2
    nCols = image_ori.shape[1]
    j = nGridsHor # j = number of existing grids
    
    for k in range(nGridsHor):
        grid_width = ROI[k].shape[1]
        grid_height = ROI[k].shape[0]
        idx_leftCol = start_column
        idx_rightCol = end_column
        for i in range(nMovements): 
            if i == nMovements - 1:
                # extrem
                grid_width =  nCols - idx_rightCol # grid width is symetrical 
            
            # left column
            ROI[j] = image_ori[idx_row : idx_row + grid_height + 1, idx_leftCol - grid_width : idx_leftCol + 1]
            # right column
            ROI[j+1] = image_ori[idx_row : idx_row + grid_height + 1, idx_rightCol : idx_rightCol + grid_width + 1] 
            
            idx_leftCol = idx_leftCol - grid_width
            idx_rightCol = idx_rightCol + grid_width

            if i == 0:
                # paint vertical lines just once 
                image_res[:, idx_leftCol] = (0,0, 255)
                image_res[:, idx_rightCol] = (0,0, 255)

            j = j + 2  

        idx_row = idx_row + grid_height  


In [6]:
# Com la columna es troba al voltant del centre d'una imatge, ens situem en el centre de cada matriu d'imatges que hem generat 
# i ens desplacem x posicions a l'esquerra i a la dreta per abastar tota la regió que pot ocupar la columna 
# x és un valor que fixo, per exemple, en 50 unitats, però que segons la imatge convé que sigui més gran o més petit per fer front a canvis 
# d'il·luminació (imatge cremada) o a columnes que no siguin rectes, que tinguin una certa rotació

# Creem una llista, images_res, que contindrà les imatges resultants i que inicialitzem a la llista d'imatges d'entrada
# Les imatges resultants seran en RGB, ja que les files i columnes de les quadrícules les pintarem en vermell 

nGridsVer = int(input("Entra el nombre de quadricules verticals que vols generar: "))
nGridsHor = int(input("Entra el nombre de quadricules horitzontals que vols generar: "))


# Creem una llista que contindrà per a cada imatge les regions d'interès 
ROI = np.zeros((nGridsVer * nGridsHor), object)

for i in range(len(images_ori)):  
    column, start_column, end_column = getColumn(images_ori[i])
    gridColumn(column, start_column, end_column, nGridsHor, images_res[i], ROI)
    gridAroundColumn(start_column, end_column, nGridsHor, nGridsVer, images_ori[i], images_res[i], ROI)
    
    cv.imwrite(f"Output_image{i}" , images_res[i])
    cv.imshow(f"Output_image {i+1}", images_res[i])
    #cv.waitKey(0)
    # Tant a la part esquerra de la columna com en la part dreta, generem 6 quadrícules 
    # de la mateixa mida entre la fila inicial i la fila central de la imatge i entre la columna inicial i la columna que pertany a l'inici de la columna vertebral
    
